In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import re
from re import sub
from decimal import Decimal
import random
import pandas as pd
from urllib.request import urlopen
import dateutil.parser
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 12]

from fake_useragent import UserAgent
import os
import json
from pymongo import MongoClient




In [2]:
#setting up fake user agent
ua = UserAgent()
user_agent = {'User-agent': ua.random}

#setting user agent names
opts=Options()
opts.add_argument('user-agent=user_agent')


In [6]:
# #first page url for yelp san francisco restaurants
# url1='https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco,+CA&start=0'   
# ##### get the url to feed into beautifulsoup
# url_list=[]
# #for i in range (page 3 to total page+1)
# for i in range(0,1000,20):
#     url_list.append(url1[:-1]+str(i))
# url_list[0]

In [2]:
# #loop through each url in the url_list to get the restaurant url data
# restaurant_url=[]
# for i in url_list:
#     #random user-agent
#     ua = UserAgent()
#     user_agent = {'User-agent': ua.random}
#     #start using beautiful soup
#     response = requests.get(i, headers=user_agent)
#     sleeptime=random.uniform(3,5)
#     time.sleep(sleeptime)
#     print(response.status_code)
#     soup=BeautifulSoup(response.text, "lxml")
#     #url for individual houses
#     for i in soup.find_all("a", {"class" :"biz-name js-analytics-click"}):
#         restaurant_url.append('https://www.yelp.com'+i['href'])


In [3]:
#restaurant_url

In [37]:
# #save restaurant_url into a csv file
# df = pd.DataFrame(restaurant_url)
# df.to_csv('restaurant_url.csv', index=False)

In [3]:
#import the csv back to a list
import csv
with open('restaurant_url.csv', 'r') as f:
    reader = csv.reader(f)
    restaurant_url = list(reader)
    
restaurant_url.remove(restaurant_url[0])

#restaurant_url


In [4]:
#delist url
restaurant_url = [y for x in restaurant_url for y in x]


In [5]:
len(restaurant_url)

1000

In [23]:
with open('restaurant_reviews.json', 'r') as f:
     restaurants = json.load(f)

In [24]:
type(restaurants), len(restaurants)

(list, 249)

## Open individual restaurants to get the name, reviews, and other information

In [25]:
#open individual restaurant's url using beautiful soup and extract all info


#restaurants=[]
#started at 241 this time, so 241 is the [0] in the data
#scraped up to 290
#next time: start from 0:100, then 100:200, 200:241
for j in restaurant_url[200:241]:
    #using random user-agent
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}
    #feed into beautiful soup
    response = requests.get(url=j, headers=user_agent)
    sleeptime=random.uniform(4,8)
    time.sleep(sleeptime)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    #print(soup)
    
    #getting individual restaurant characteristics
    restaurant={}
    
    #name
    name = soup.find('h1').text
    name = name.strip()
    
    restaurant['name']=name
    
    #overall rating star, overall # of review
    info = soup.find_all('div',{"class":"biz-rating biz-rating-very-large clearfix"})
    info = str(info)
    info1 = info.split('review')
    
    sub_info1 = info1[0].split('title="')
    rating = sub_info1[1][0:3]
    
    sub_info2 = info1[1].split('\n')
    review_num = int(sub_info2[1].replace(' ',''))

    restaurant['overall-rating']=rating
    restaurant['reviews-total-number']=review_num
 
    review_all=[]
    #detailed reviews
    #1st page reviews
    reviews = soup.find_all('p',{"lang":"en"})
    review_all.append(reviews)
    #pages 2-end reviews
    for k in range(20,(int(review_num/20)+1)*20, 20):
        print(k)
        #using random user-agent
        ua = UserAgent()
        user_agent = {'User-agent': ua.random}
        #feed into beautiful soup
        response = requests.get(url=j.split("?")[0]+'?start='+str(k), headers=user_agent)
        sleeptime=random.uniform(5,7)
        time.sleep(sleeptime)
        print(response.status_code)
        page = response.text
        soup = BeautifulSoup(page,"lxml")

        reviews = soup.find_all('p',{"lang":"en"})
        review_all.append(reviews)

    #delist the review_all list
    review_all=sum(review_all, [])
    #turn each review into a string so that it can be added to mongoDB
    text_reviews_all=[str(x) for x in review_all]
        
    restaurant['text_review']=text_reviews_all
    
    restaurants.append(restaurant)

        
        


200
20
200
40
200
60
200
80
200
100
200
120
200
140
200
160
200
180
200
200
200
220
200
240
200
260
200
280
200
300
200
320
200
340
200
360
200
380
200
400
200
420
200
440
200
460
200
480
200
500
200
520
200
540
200
200
20
200
40
200
60
200
80
200
100
200
120
200
140
200
160
200
180
200
200
200
220
200
240
200
260
200
280
200
300
200
320
200
340
200
360
200
380
200
400
200
420
200
440
200
460
200
480
200
500
200
520
200
540
200
560
200
580
200
600
200
620
200
640
200
660
200
680
200
700
200
720
200
740
200
760
200
780
200
800
200
820
200
840
200
860
200
880
200
900
200
920
200
940
200
960
200
980
200
1000
200
1020
200
1040
200
1060
200
1080
200
1100
200
1120
200
1140
200
1160
200
1180
200
1200
200
1220
200
1240
200
1260
200
1280
200
1300
200
1320
200
1340
200
1360
200
1380
200
1400
200
1420
200
1440
200
1460
200
1480
200
1500
200
1520
200
1540
200
1560
200
1580
200
1600
200
1620
200
1640
200
1660
200
1680
200
1700
200
1720
200
1740
200
1760
200
1780
200
1800
200
1820
200
1840
200
1860


In [26]:
len(restaurants)

290

In [10]:
restaurants[-1]

{'name': 'Farmhouse Kitchen Thai Cuisine',
 'overall-rating': '4.0',
 'reviews-total-number': 805,
 'text_review': ['<p lang="en">Oh \xa0man I love the food for this place. I ordered the fried chicken and it was absolutely delicious. The plate comes with blue rice (similar texture to basmati), a really large piece of chicken (with a sort of sweet and sour sauce) plus naan and curry. Everything on the plate compliments the other sections. It\'s $22 but well worth and easily split into two meals (which is what i did). DEFINITELY REC THIS DISH.<br/><br/>Boyfriend ordered the fried rice and that was also delicious for $15. Ingredients were fresh and great, but I think I\'m more willing to pay for lesser quality at a lower price.</p>',
  '<p lang="en">I should preface this review by saying that I\'m not a big fan of Thai food. Farmhouse Kitchen is definitely an exception though! I\'ve had both lunch and dinner here and enjoyed everything. The food and decor is pretty unique so at the very l

In [27]:
with open('restaurant_reviews.json', 'w') as f:
     json.dump(restaurants, f)

In [13]:
restaurant_url[153]

'https://www.yelp.com/biz/farmhouse-kitchen-thai-cuisine-san-francisco?osq=Restaurants'